# Texas Cosmetologist Violations

Texas has a system for [searching for license violations](https://www.tdlr.texas.gov/cimsfo/fosearch.asp). You're going to search for cosmetologists!

> You can use the classwork notebook and also [my Everything scraping reference](https://jonathansoma.com/everything/scraping)

I shouldn't have to say this, but **do not use ChatGPT for this assignment.**

## Setup: Import what you'll need to scrape the page

We'll be using either Playwrightfor this, *not* requests.

In [ ]:
from playwright.async_api import async_playwright
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless=True)
page = await browser.new_page()
await page.goto("https://www.tdlr.texas.gov/cimsfo/fosearch.asp")
from bs4 import BeautifulSoup

## Starting your search

Starting from [here](https://www.tdlr.texas.gov/cimsfo/fosearch.asp), search for cosmetologist violations for people with the last name **Nguyen**.

In [ ]:
#I have been getting time error despite putting the correct surname in for at least 30 times!!! PLease help 
from playwright.async_api import async_playwright
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless=False)
page = await browser.new_page()
await page.goto("https://www.tdlr.texas.gov/cimsfo/fosearch.asp")
from bs4 import BeautifulSoup
await page.get_by_label("Search by License Program Type").select_option('Cosmetologist')
await page.fill("input[id='pht_lnm']", "Nguyen")
await page.get_by_role("button", name="Search").click()
html = await page.content()
from bs4 import BeautifulSoup
soup_doc = BeautifulSoup(html, "html.parser")
table = soup_doc.find("div", class_="main-content")
if table:
    print("Results Found:")
    print(table.prettify())
else:
    print("No results found.")
print(table)

In [97]:
# #And then I got Server Error

# 404 - File or directory not found.
# The resource you are looking for might have been removed, had its name changed, or is temporarily unavailable.

In [99]:
page = await browser.new_page()
await page.goto("https://www.tdlr.texas.gov/cimsfo/fosearch.asp")
await page.get_by_label("Search by Last Name").fill('Nguyen')
await page.locator("input[name=\"B1\"]").click()

In [100]:
import io 
import pandas as pd
html = await page.content()
tables = pd.read_html(io.StringIO(html))
print(tables[0])

                                    Name and Location  \
0   NGUYEN, ELLIE UYEN City: HUMBLE  County: HARRI...   
1   NGUYEN, PHUC City: HOUSTON  County: HARRIS  Zi...   
2   NGUYEN, TRONG City: FORT WORTH  County: TARRAN...   
3   NGUYEN, SA City: AUSTIN  County: TRAVIS  Zip C...   
4   NGUYEN, LYNH City: RICHARDSON  County: DALLAS ...   
..                                                ...   
78  NGUYEN, KELLY PHUONG N  City: GARDEN GROVE  Co...   
79  NGUYEN, QUOC BAO City: ARLINGTON  County: TARR...   
80  NGUYEN, DANIEL THAO City: PLANO  County: COLLI...   
81  NGUYEN, THANH THU City: EL PASO  County: EL PA...   
82  NGUYEN, CINDY City: TEMPLE  County: BELL  Zip ...   

                                                Order  \
0   Date: 10/30/2024 Respondents Ellie Uyen Nguyen...   
1   Date: 10/9/2024 Respondent is assessed an admi...   
2   Date: 9/18/2024 Respondents Hanh Ngo, Linh Ngo...   
3   Date: 8/26/2024 Respondent is assessed an admi...   
4   Date: 8/8/2024 Respondent 

## Scraping

Once you are on the results page, do this. **I step you through things bit by bit, so it's going to be a little different than we did in class.** Also, no `pd.read_html` allowed because this isn't reaaallly tabular data (or there's just too much to clean up).

Once you've loaded the page: **what selector are you going to do use to find the data?** In class we used things like `.article` or `.head-list`.

> You honestly can do this all with Playwright, no BeautifulSoup involved! But we didn't cover that in class so you should stick to this method:
> 
> ```python
> html = await page.content()
> doc = BeautifulSoup(html)
> ```

In [101]:
html = await page.content()
doc = BeautifulSoup(html)

### Grab the rows, loop through each result and print the entire row

It's probably too many rows to do all at once: once you have a list, use `[:10]` to only show the first ten! For example, if you saved the table rows into `results` you might do something like this:

```python
for result in results[:10]:
    print("------")
    print(result)
```

Although you'd want to print out the text from the row (I give example output below).

In [102]:
results = tables[0].values.tolist()
for result in results[:10]:
    print("------")
    print(result)

------
['NGUYEN, ELLIE UYEN City: HUMBLE  County: HARRIS  Zip Code: 77346  License #: 798636 Complaint # COS20240003068', 'Date: 10/30/2024 Respondents Ellie Uyen Nguyen and Bao Q. Truong are assessed an administrative penalty in the amount of $3,750.', "Respondents failed to clean, disinfect, and sterilize metal instruments with a Department-approved sterilizer in accordance with the sterilizer or sanitizers manufacturer's instructions; Respondents failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foot spa was not used; Respondents failed to clean and disinfect all wax pots; Respondents failed to clean diamond, carbide, natural and metal bits after each use with a brush or ultrasonic cleaner, or by immersing in acetone; Respondents failed to dispose of single use items after each use."]
------
['NGUYEN, PHUC City: HOUSTON  County: HARRIS  Zip Code: 77083  License #: 807327 Complaint # COS20230005653', 'Date: 10/9/2024 Respondent is as

The result should look something like this:

```
Name and Location Order Basis for Order
NGUYEN, THANH
City: FRISCO
County: COLLIN
Zip Code: 75034


License #: 790672

Complaint # COS20210004784 Date: 11/16/2021

Respondent is assessed an administrative penalty in the amount of $1,875. Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day, the Department is charging 2 violations; Respondent operated a cosmetology salon without the appropriate license.
NGUYEN, LONG D
City: SAN SABA
County: SAN SABA
Zip Code: 76877
```

### Loop through each result and print each person's name

You'll probably get an error because the first one doesn't have a name. How do you make that not happen?! If you want to ignore an error, you use code like this:

```python
try:
   # try to do something
except:
   print("It didn't work')
```

It should help you out. If you don't want to print anything when there's an error, you can type `pass` instead of the `print` statement.

**Why doesn't the first one have a name?**

Output should look like this:

```
Doesn't have a name
NGUYEN, THANH
NGUYEN, LONG D
NGUYEN, LUCIE HUONG
NGUYEN, CHINH
NGUYEN, JIMMY
```

* *Tip: The name has a class you can use. The class name is reused in a lot of places, but because it's the first one you don't have to worry about that!*
* *Tip: Instead of searching across the entire page – like `doc.select_one` – you should be doing your searching just inside of each **row** (I used this technique in the beginning of class with BeautifulSoup when we were scraping the books page)* 

In [127]:
from bs4 import BeautifulSoup
html = await page.content()

soup_doc = BeautifulSoup(html, "html.parser")

rows = soup_doc.select("tr")

for row in rows[:10]:
    try:
        name = row.select_one("span.results_text")
        if name: 
            print(name.text.strip())
        else:
            print("No name found")
    except:
        print("Error:", e)

No name found
NGUYEN, ELLIE UYEN
NGUYEN, PHUC
NGUYEN, TRONG
NGUYEN, SA
NGUYEN, LYNH
NGUYEN, LISA
NGUYEN, CHAU MINH
NGUYEN, KEVIN C
NGUYEN, THANH D


In [126]:
# results = soup_doc.find_all("span", class_="results_text")
# results

## Loop through each result, printing each violation description ("Basis for order")

Your results should look something like:

```
Doesn't have a violation
Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day, the Department is charging 2 violations; Respondent operated a cosmetology salon without the appropriate license.
Respondent failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foot spa was not used, the Department is charging 2 violations; Respondent failed to clean, disinfect, and sterilize manicure and pedicure implements after each use; Respondent failed to clean and disinfect manicure tables prior to use for each client.
...
```

> - *Tip: You'll get an error even if you're ALMOST right - which row is causing the problem?*
> - *Tip: Or I guess you could just skip the one with the problem using try/except...*

In [132]:
for row in rows[:10]:
    try:
        complaint_td = row.find_all("td")[-1]
        if complaint_td: 
            complaint = complaint_td.text.strip()
            print(complaint)
        else:
            print("Doesn't have a complaint")
    except:
        print("Error: CHECKKK!!!")

Error: CHECKKK!!!
Respondents failed to clean, disinfect, and sterilize metal instruments with a Department-approved sterilizer in accordance with the sterilizer or sanitizers manufacturer's instructions; Respondents failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foot spa was not used; Respondents failed to clean and disinfect all wax pots; Respondents failed to clean diamond, carbide, natural and metal bits after each use with a brush or ultrasonic cleaner, or by immersing in acetone; Respondents failed to dispose of single use items after each use.
Respondent failed to follow whirlpool foot spas cleaning and sanitization procedures as required daily, bi-weekly, and before use by each patron.
Respondents failed to follow whirlpool foot spas cleaning and sanitization procedures as required; Respondents failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foot spa was not used; Responde

## Loop through each result, printing the complaint number

Output should look similar to this:

```
Doesn't have a complaint number
COS20210004784
COS20210009745
COS20210011484
...
```

- *Tip: Think about the order of the elements. Can you count from the opposite direction than you normally do?*

In [138]:
for row in rows[:10]:
    try:
        complaint_label = row.select_one("span.default_text:contains('Complaint #')")
        if complaint_label:
            complaint_number = complaint_label.find_next_sibling("span", class_="results_text")
            if complaint_number:
                print(complaint_number.text.strip())
            else:
                print("no complaint")
    except:
        print("Error: CHECKKK!!!")

COS20240003068
COS20230005653
COS20240011518
COS20240008640
COS20230004635
COS20230010686
COS20220007736
COS20230020006
COS20240000594


## Saving the results

### Loop through each result to create a list of dictionaries

Each dictionary must contain

- Person's name
- Violation description
- Violation number
- License Numbers
- Zip Code
- County
- City

Create a new dictionary for each result (except the header).

Based on what you print out, the output might look something like:

```
This row is broken: Name and Location Order Basis for Order
{'name': 'NGUYEN, THANH', 'city': 'FRISCO', 'county': 'COLLIN', 'zip_code': '75034', 'complaint_no': 'COS20210004784', 'license_numbers': '790672', 'complaint': 'Respondent failed to clean and sanitize whirlpool foot spas as required at the end of each day, the Department is charging 2 violations; Respondent operated a cosmetology salon without the appropriate license.'}
{'name': 'NGUYEN, LONG D', 'city': 'SAN SABA', 'county': 'SAN SABA', 'zip_code': '76877', 'complaint_no': 'COS20210009745', 'license_numbers': '760420, 1620583', 'complaint': 'Respondent failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foot spa was not used, the Department is charging 2 violations; Respondent failed to clean, disinfect, and sterilize manicure and pedicure implements after each use; Respondent failed to clean and disinfect manicure tables prior to use for each client.'}
```

> _**Tip:** Depending on how you do this, you might want to figure out how to search for "next siblings" or "following siblings"_

In [154]:
results = []
for row in rows[1:]: #Skipping header 
    try: 
        # name_span = row.select_one("span.results_text")
        # name = name_span.text.strip() if name_span else "No name"
        
        # city_span = row.select("span.results_text")[1] 
        # city = city_span.text.strip() if city_span else "No city"
        
        # county_span = row.select("span.results_text")[3] 
        # county = county_span.text.strip() if county_span else "No county"

        # zip_code_span = row.select("span.results_text")[5]
        # zip_code = zip_code_span.text.strip() if zip_code_span else "No zip code"

        # license_span = row.select("span.results_text")[7] 
        # license_numbers = license_span.text.strip() if license_span else "No license numbers"

        # complaint_no_span = row.select("span.results_text")[9] 
        # complaint_no = complaint_no_span.text.strip() if complaint_no_span else "No complaint number"

        # complaint_td = row.find_all("td")[-1] 
        # complaint = complaint_td.text.strip() if complaint_td else "No complaint description"

        name_span = row.select_one("span.results_text")
        name = name_span.text.strip() if name_span else "No name"

        spans = row.select("span.results_text")
        city = spans[1].text.strip() if len(spans) > 1 else "No city"
        county = spans[3].text.strip() if len(spans) > 3 else "No county"
        zip_code = spans[5].text.strip() if len(spans) > 5 else "No zip code"
        license_numbers = spans[7].text.strip() if len(spans) > 7 else "No license numbers"
        complaint_no = spans[9].text.strip() if len(spans) > 9 else "No complaint number"
        complaint_td = row.find_all("td")[-1]  
        complaint = complaint_td.text.strip() if complaint_td else "No complaint description"
        
        result = {
            "name": name,
            "city": city,
            "county": county,
            "zip_code": zip_code,
            "license_numbers": license_numbers,
            "complaint_no": complaint_no,
            "complaint": complaint,
        }
        results.append(result) 
    except:
        print(f"This row is broken: {row.text.strip()}")
        print("Error: CHECKKK!!!")

for result in results:
    print(result)

{'name': 'NGUYEN, ELLIE UYEN', 'city': 'HUMBLE', 'county': '77346', 'zip_code': 'COS20240003068', 'license_numbers': 'No license numbers', 'complaint_no': 'No complaint number', 'complaint': "Respondents failed to clean, disinfect, and sterilize metal instruments with a Department-approved sterilizer in accordance with the sterilizer or sanitizers manufacturer's instructions; Respondents failed to keep a record of the date and time of each foot spa daily or bi-weekly cleaning and if the foot spa was not used; Respondents failed to clean and disinfect all wax pots; Respondents failed to clean diamond, carbide, natural and metal bits after each use with a brush or ultrasonic cleaner, or by immersing in acetone; Respondents failed to dispose of single use items after each use."}
{'name': 'NGUYEN, PHUC', 'city': 'HOUSTON', 'county': '77083', 'zip_code': 'COS20230005653', 'license_numbers': 'No license numbers', 'complaint_no': 'No complaint number', 'complaint': 'Respondent failed to follo

### Save that to a CSV named `output.csv`

The dataframe should look something like...

|index|name|city|county|zip_code|complaint_no|license_numbers|complaint|
|---|---|---|---|---|---|---|---|
|0|NGUYEN, THANH|FRISCO|COLLIN|75034|COS20210004784|790672|Respondent failed to clean and sanitize whirlp...|
|1|NGUYEN, LONG D|SAN SABA|SAN SABA|76877|COS20210009745|760420, 1620583|Respondent failed to keep a record of the date...|


- *Tip: If you send a list of dictionaries to `pd.DataFrame(...)`, it will create a dataframe out of that list!*

In [ ]:
import pandas as pd 

In [158]:
df = pd.DataFrame(results)
output_file = 'texas_output.csv'
df.to_csv(output_file, index=False)

### Open the CSV file and examine the first few. Make sure you didn't save an extra weird unnamed column.

In [160]:
pd.read_csv('texas_output.csv')

,name,city,county,zip_code,license_numbers,complaint_no,complaint
0,"NGUYEN, ELLIE UYEN",HUMBLE,77346,COS20240003068,No license numbers,No complaint number,"Respondents failed to clean, disinfect, and st..."
1,"NGUYEN, PHUC",HOUSTON,77083,COS20230005653,No license numbers,No complaint number,Respondent failed to follow whirlpool foot spa...
2,"NGUYEN, TRONG",FORT WORTH,76177,COS20240011518,No license numbers,No complaint number,Respondents failed to follow whirlpool foot sp...
3,"NGUYEN, SA",AUSTIN,78729,COS20240008640,No license numbers,No complaint number,Respondent failed to follow whirlpool foot spa...
4,"NGUYEN, LYNH",RICHARDSON,75081,COS20230004635,No license numbers,No complaint number,Respondent failed to follow whirlpool foot spa...
...,...,...,...,...,...,...,...
78,"NGUYEN, KELLY PHUONG N",GARDEN GROVE,92843,COS20220006797,No license numbers,No complaint number,Respondent failed to keep a record of the date...
79,"NGUYEN, QUOC BAO",ARLINGTON,76018,COS20210009267,No license numbers,No complaint number,Respondent failed to follow whirlpool foot spa...
80,"NGUYEN, DANIEL THAO",PLANO,75093,COS20220004501,No license numbers,No complaint number,Respondent failed to clean and sanitize whirlp...
81,"NGUYEN, THANH THU",EL PASO,79904,COS20210012742,No license numbers,No complaint number,Respondent failed to clean and sanitize whirlp...
